<a href="https://colab.research.google.com/github/NarcisoHC/dotfiles/blob/master/sentiment_analysis_with_word2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sentiment analysis with Word2Vec

### Exercise objectives:
- Convert words to vectors with Word2Vec
- Use the word representation given by Word2vec to feed a RNN

<hr>
<hr>


# The data


❓ **Question** ❓ Let's first load the data. You don't have to understand what is going on in the function, it does not matter here.

⚠️ **Warning** ⚠️ The `load_data` function has a `percentage_of_sentences` argument. Depending on your computer, there are chances that a too large number of sentences will make your compute slow down, or even freeze - your RAM can even overflow. For that reason, **you should start with 10% of the sentences** and see if your computer handles it. Otherwise, rerun with a lower number. 

⚠️ **DISCLAIMER** ⚠️ **No need to play _who has the biggest_ (RAM) !** The idea is to get to run your models quickly to prototype. Even in real life, it is recommended that you start with a subset of your data to loop and debug quickly. So increase the number only if you are into getting the best accuracy.

In [5]:
###########################################
### Just run this cell to load the data ###
###########################################

import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.text import text_to_word_sequence

def load_data(percentage_of_sentences=None):
    train_data, test_data = tfds.load(name="imdb_reviews", split=["train", "test"], batch_size=-1, as_supervised=True)

    train_sentences, y_train = tfds.as_numpy(train_data)
    test_sentences, y_test = tfds.as_numpy(test_data)
    
    # Take only a given percentage of the entire data
    if percentage_of_sentences is not None:
        assert(percentage_of_sentences> 0 and percentage_of_sentences<=100)
        
        len_train = int(percentage_of_sentences/100*len(train_sentences))
        train_sentences, y_train = train_sentences[:len_train], y_train[:len_train]
  
        len_test = int(percentage_of_sentences/100*len(test_sentences))
        test_sentences, y_test = test_sentences[:len_test], y_test[:len_test]
    
    X_train = [text_to_word_sequence(_.decode("utf-8")) for _ in train_sentences]
    X_test = [text_to_word_sequence(_.decode("utf-8")) for _ in test_sentences]
    
    return X_train, y_train, X_test, y_test

X_train, y_train, X_test, y_test = load_data(percentage_of_sentences=10)

In the previous exercise, you trained a Word2vec representation and converted all your training sentences in order to feed them into a RNN, as shown in the first step of this Figure : 

<img src="word2vec_representation.png" width="400px" />



❓ **Question** ❓ Here, let's re-do exactly what you have done in the previous exercise. First, train a word2vec model (with the arguments that you want) on your training sentence. Store it into the `word2vec` variable.

In [7]:
from gensim.models import Word2Vec

word2vec = Word2Vec(sentences=X_train, min_count=3, window=8)

In [10]:
word2vec.wv['dog'].shape

(100,)

Let's reuse the functions of the previous exercise to convert your training and test data into something you can fed into a RNN.

❓ **Question** ❓ Read the following function to be sure you understand what is going on, and run it.

In [11]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

# Function to convert a sentence (list of words) into a matrix representing the words in the embedding space
def embed_sentence(word2vec, sentence):
    embedded_sentence = []
    for word in sentence:
        if word in word2vec.wv:
            embedded_sentence.append(word2vec.wv[word])
        
    return np.array(embedded_sentence)

# Function that converts a list of sentences into a list of matrices
def embedding(word2vec, sentences):
    embed = []
    
    for sentence in sentences:
        embedded_sentence = embed_sentence(word2vec, sentence)
        embed.append(embedded_sentence)
        
    return embed

# Embed the training and test sentences
X_train_embed = embedding(word2vec, X_train)
X_test_embed = embedding(word2vec, X_test)


# Pad the training and test embedded sentence

In [12]:
X_train_pad = pad_sequences(X_train_embed, dtype='float32', padding='post', maxlen=200, value=-1000)
X_test_pad = pad_sequences(X_test_embed, dtype='float32', padding='post', maxlen=200, value=-1000)

☝️ To be sure that it worked, let's check the following for `X_train_pad` and `X_test_pad` :
- they are numpy arrays
- they are 3-dimensional
- the last dimension is of the size of your word2vec embedding space (you can get it with `word2vec.wv.vector_size`
- the first dimension is of the size of your `X_train` and `X_test`

✅ **Good Practice** ✅ Such tests are quite important! Not only in this exercise, but in real-life applications. It prevents from searching at errors too late and from letting them propagate through the entire notebook.

In [13]:
# TEST ME
for X in [X_train_pad, X_test_pad]:
    assert type(X) == np.ndarray
    assert X.shape[-1] == word2vec.wv.vector_size


assert X_train_pad.shape[0] == len(X_train)
assert X_test_pad.shape[0] == len(X_test)

# Baseline model

It is always good to have a very simple model to test your own model against - to be sure you do something better than a very simple algorithm.

❓ **Question** ❓ What is your baseline accuracy? In this case, your baseline can be to predict the label that is the most present in `y_train` (of course, if the dataset is balanced, the baseline accuracy is 1/n where n is the number of classes - 2 here).

In [14]:
ones = y_train.sum()
zeros = y_train.shape[0]-ones

In [15]:
freqones = ones/y_train.shape[0]
freqzer = zeros/y_train.shape[0]
print(f'FREQUENCIES: ones= {freqones}, zeroes={freqzer}')

FREQUENCIES: ones= 0.494, zeroes=0.506


# The model

❓ **Question** ❓ Write a RNN with the following layers:
- a `Masking` layer
- a `LSTM` with 20 units and `tanh` activation function
- a `Dense` with 10 units
- an output layer that depends on your task

Then, compile your model (we advise you to use the `rmsprop` as the optimizer - at least to begin with)

In [16]:
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras.layers import Masking

def init_model(): 
    model = models.Sequential()
    model.add(Masking(mask_value=-1000))
    model.add(layers.LSTM(20, activation='tanh'))
    model.add(layers.Dense(10, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics='accuracy')
    return model

In [18]:
from tensorflow.keras.callbacks import EarlyStopping
es = EarlyStopping(patience=5, restore_best_weights=True)

model = init_model()
model.fit(X_train_pad, y_train, epochs=10, batch_size=64, validation_split=0.3, callbacks=[es])

Epoch 1/10
28/28 [==============================] - 6s 63ms/step - loss: 0.6941 - accuracy: 0.5388 - val_loss: 0.6790 - val_accuracy: 0.6013
Epoch 2/10
28/28 [==============================] - 1s 20ms/step - loss: 0.6800 - accuracy: 0.5787 - val_loss: 0.6794 - val_accuracy: 0.5733
Epoch 3/10
28/28 [==============================] - 1s 20ms/step - loss: 0.6708 - accuracy: 0.5987 - val_loss: 0.6601 - val_accuracy: 0.6147
Epoch 4/10
28/28 [==============================] - 1s 20ms/step - loss: 0.6656 - accuracy: 0.6030 - val_loss: 0.6557 - val_accuracy: 0.6213
Epoch 5/10
28/28 [==============================] - 1s 19ms/step - loss: 0.6484 - accuracy: 0.6339 - val_loss: 0.7118 - val_accuracy: 0.5093
Epoch 6/10
28/28 [==============================] - 1s 19ms/step - loss: 0.6515 - accuracy: 0.6176 - val_loss: 0.6655 - val_accuracy: 0.6067
Epoch 7/10
28/28 [==============================] - 1s 19ms/step - loss: 0.6482 - accuracy: 0.6238 - val_loss: 0.6354 - val_accuracy: 0.6520
Epoch 8/10
28

❓ **Question** ❓ Fit the model on your embedded and padded data - do not forget the early stopping criterion.

❗ **Remark** ❗ Your accuracy with greatly depend on your training test corpus. Here just make sure that your performance is above the baseline model (which should be the case even if you loaded only 20% of the initial IMDB data).

In [ ]:
# YOUR CODE HERE

❓ **Question** ❓ Evaluate your model on the test set

In [19]:
model.evaluate(X_test_pad, y_test)

79/79 [==============================] - 1s 8ms/step - loss: 0.7156 - accuracy: 0.5520


[0.7155839204788208, 0.5519999861717224]

# Trained Word2Vec - Transfer Learning

Your accuracy, while above the baseline model, might be quite low. There are multiple options to improve it, as data cleaning and improving the quality of the embedding.

We won't dig into data cleaning strategies here. On the other hand, let's try to improve the quality of our embedding. But instead of just loading a larger corpus, why not benefiting from the embedding that other have learnt? Because, the quality of an embedding, i.e. the proximity of the words, can be derived from different tasks. This is exactly what transfer learning is.



❓ **Question** ❓ List all the different models available in the word2vec thanks to this : 

In [20]:
import gensim.downloader as api
print(list(api.info()['models'].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


❓ **Question** ❓ Load one of the pre-trained word2vec embedding spaces. 

You can do that with `api.load(the-model-of-your-choice)`, and store it in `word2vec_transfer`

In [21]:
word2vec_transfer = api.load('glove-twitter-200')

[==================================================] 100.0% 758.5/758.5MB downloaded


❓ **Question** ❓ Check the size of the vocabulary, but also the size of the embedding space.

In [23]:
word2vec_transfer.wv.vectors.shape

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


(1193514, 200)

❓ **Question** ❓ Let's embedd `X_train` and `X_test`, as in the first question where we provided the functions to so ! (There is a slight different in the `embed_sentence_with_TF` function that we will not dig into)

In [24]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

# Function to convert a sentence (list of words) into a matrix representing the words in the embedding space
def embed_sentence_with_TF(word2vec, sentence):
    embedded_sentence = []
    for word in sentence:
        if word in word2vec:
            embedded_sentence.append(word2vec[word])
        
    return np.array(embedded_sentence)

# Function that converts a list of sentences into a list of matrices
def embedding(word2vec, sentences):
    embed = []
    
    for sentence in sentences:
        embedded_sentence = embed_sentence_with_TF(word2vec, sentence)
        embed.append(embedded_sentence)
        
    return embed

# Embed the training and test sentences
X_train_embed_2 = embedding(word2vec_transfer, X_train)
X_test_embed_2 = embedding(word2vec_transfer, X_test)

❓ **Question** ❓  Do not forget to pad your results and store it in `X_train_pad_2` and `X_test_pad_2`.

In [25]:
X_train_pad_2 = pad_sequences(X_train_embed_2, dtype='float32', padding='post', maxlen=200, value=-1000)
X_test_pad_2 = pad_sequences(X_test_embed_2, dtype='float32', padding='post', maxlen=200, value=-1000)

❓ **Question** ❓ Reinitialize a model and fit it on your new embedded (and padded) data!  Evaluate it on your test set and compare it to your previous accuracy.

❗ **Remark** ❗ The training could take some time here. You can just compute 10 epochs (this is **not** a good practice, it is just not to wait too long) and go to the next exercise while it trains - or take a break, you probably deserve it ;)

In [29]:
es = EarlyStopping(patience=5, restore_best_weights=True)

model = init_model()
model.fit(X_train_pad_2, y_train, epochs=20, batch_size=64, validation_split=0.3, callbacks=[es])

Epoch 1/20
28/28 [==============================] - 6s 65ms/step - loss: 0.6860 - accuracy: 0.5404 - val_loss: 0.6994 - val_accuracy: 0.5427
Epoch 2/20
28/28 [==============================] - 1s 22ms/step - loss: 0.6308 - accuracy: 0.6419 - val_loss: 0.6553 - val_accuracy: 0.5933
Epoch 3/20
28/28 [==============================] - 1s 23ms/step - loss: 0.5987 - accuracy: 0.6846 - val_loss: 0.6645 - val_accuracy: 0.6227
Epoch 4/20
28/28 [==============================] - 1s 23ms/step - loss: 0.5656 - accuracy: 0.7191 - val_loss: 0.5650 - val_accuracy: 0.7133
Epoch 5/20
28/28 [==============================] - 1s 22ms/step - loss: 0.5252 - accuracy: 0.7412 - val_loss: 0.5746 - val_accuracy: 0.7067
Epoch 6/20
28/28 [==============================] - 1s 23ms/step - loss: 0.5262 - accuracy: 0.7460 - val_loss: 0.9525 - val_accuracy: 0.5027
Epoch 7/20
28/28 [==============================] - 1s 22ms/step - loss: 0.5881 - accuracy: 0.6998 - val_loss: 0.5233 - val_accuracy: 0.7573
Epoch 8/20
28

In [30]:
res = model.evaluate(X_test_pad_2, y_test, verbose=0)

print(f'The accuracy evaluated on the test set is of {res[1]*100:.3f}%')

The accuracy evaluated on the test set is of 79.240%


Because your new word2vec has been trained on a large corpus, it has a representation for many many words! Way more than with your small dataset, especially as you discarder words that were not present more than a given number of time in the train set. For that reason, you have way more embedded words in your train and test set, which makes each iteration longer than previously